# __Explore different threshold assessments__
compares first 25s __Labview__ file with 25 to 30s from __EEG__ file 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

rcParams['figure.figsize'] = 10, 10
sns.set_style("whitegrid")

# sampling rate 10kHz
sr = 10000

# take every N-th row (for skip_rows)
n=2
n_sr=sr/n

# subject to analyze
ID = 20
f_string = f'../DATA/Complete/Res_Cadence_P{ID}'

/Users/sebastienvillard/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


## Load EEG file
read and downsample to 5kHz

In [ ]:
file_name_Eeg = f_string+'Lite.txt'

# count the lines
num_lines = sum(1 for l in open(file_name_Eeg))

# the row indices to skip - make sure 0 is not included to keep header
skip_idx = [x for x in range (1, num_lines) if x % n != 0]

# read data
eeg_df = pd.read_csv(file_name_Eeg,
                     usecols=['Time', 'Fr', 'B', 'MF'],
                     skiprows= skip_idx
                    )

## MF correction for distance + convertion to mT
coef_distance = 1.27 #  from 16cm to 0cm
coef_convert = 20 #  from V to mT (see probe manual)

eeg_df['MF'] = eeg_df['MF']*2*coef_distance*coef_convert # *2 because only half signal on high level input

eeg_df.head()

In [ ]:
plt.figure(1)
plt.plot(eeg_df['Time'], eeg_df['MF'])
subset = eeg_df['Time'].diff()>1
subset[0]=True
plt.plot(eeg_df[subset]['Time'], eeg_df[subset]['MF'],'ro')
# plt.plot(eeg_df["Time"])

## Load Labview file
read and downsample to 5kHz

In [3]:
file_name_Lab = f_string+'.txt'

# count the lines
header_end = 23
num_lines = sum(1 for i, l in enumerate(open(file_name_Lab)) if i > header_end)

# the row indices to skip - make sure 0 is not included to keep header
skip_idx = list(range(header_end-1))
skip_idx.extend([x for x in range (header_end, num_lines) if (x-header_end) % n != 0])
lab_df = pd.read_csv(file_name_Lab,
                     skiprows=skip_idx, 
                     delimiter='\t',
                     usecols=['Untitled', 'Untitled 1', 'Untitled 2', 'Current'])
lab_df.columns=['Time', 'MF', 'Dial', 'Current']
lab_df.head()

,Time,MF,Dial,Current
0,0.0,0.228104,1.275271e-12,0.002542
1,NaN,0.416170,4.605991e-11,-0.000677
2,NaN,0.129983,3.088293e-10,0.008015
3,NaN,0.342579,1.027759e-09,0.017028
4,NaN,0.416170,2.432673e-09,0.004474


In [58]:
plt.figure(3)
plt.plot(lab_df['MF'])
plt.plot(lab_df.query('Time == 0')['MF'], 'ro')


## __Rebuilding Signal__ 25s Labview + 5s EEG (for Time and MF)

In [ ]:
stLab = lab_df.query('Time == 0')
subset = eeg_df['Time'].diff().fillna(2)>1
stEeg = eeg_df[subset]

In [ ]:
n_MF = pd.DataFrame()

for i in range(50):
    idxLab = stLab.index.tolist()
    idxLab.append(lab_df.index[-1])
    
    idxEeg = stEeg.index.tolist()
    idxEeg.append(eeg_df.index[-1])
    
    section_Lab = np.arange(idxLab[:-1][i],idxLab[1:][i])
    section_Eeg = np.arange(idxEeg[:-1][i],idxEeg[1:][i])
    
    data = pd.concat([lab_df.loc[section_Lab,'MF'],eeg_df.loc[section_Eeg, 'MF']], 
                     keys=['LAB', 'EEG'], 
                     names=['Source', 'Iteration_no'])
    data = data.reset_index()
    data['Time']=np.linspace(start=1/n_sr, stop=len(data)*1/n_sr, num=len(data))
    data['Iteration_no'] = f'Iter{i}'

    n_MF = n_MF.append(data)
    

## Graph all iterations (__caution__) 

In [ ]:
for it in n_MF['Iteration_no'].unique():
    plt.figure()
    sns.lineplot(data = n_MF.query(f'Iteration_no == "{it}"'),
                 x='Time',
                 y='MF',
                 hue='Source'
                )
    plt.title(f'Iteration # {it}')
    plt.legend(loc='upper left')

## Table of rms value LAB vs EEG (for MF in mT from probe @ 16cm)
0.5s of LAB vs 5s of EEG

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
rms_RES = n_MF.query('Time > 24.5').groupby(by=['Source', 'Iteration_no']).apply(lambda x: np.sqrt(np.mean(x**2)))
cm = sns.light_palette("green", as_cmap=True)
rms_RES.sort_index(level=1).style.background_gradient(cmap=cm, subset=['MF'])

### graph individual Iteration based on table

In [ ]:
IT_no = 34
ax = n_MF.query(f'Source == "LAB" & Iteration_no=="Iter{IT_no}"').plot(x='Time', y='MF')
n_MF.query(f'Source == "EEG" & Iteration_no=="Iter{IT_no}"').plot(x='Time', y='MF', ax=ax)
plt.title(f'Subject ID: {ID} --- Iteration # {IT_no}')